In [1]:
# -*- encoding:utf-8 -*-
#-*- coding: cp950 -*-

import os
import sys
import itertools

import pyspark
import random

import matplotlib.pyplot as plt
import numpy as np
import numpy
from matplotlib import cm
import pandas as pd
from numpy import concatenate
from pandas import concat

from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from pyspark.sql.functions import col  
import pyspark.sql.types
from pyspark.sql.types import DoubleType
from pyspark.sql import Row
from pyspark.sql.types import Row
from pyspark.sql.functions import udf

from pyspark import since, SparkContext, keyword_only

from pyspark.ml.common import _java2py, _py2java
from pyspark.ml.wrapper import _jvm
from pyspark.ml.util import *
from pyspark.ml.wrapper import JavaEstimator, JavaModel
from pyspark.ml.param.shared import *
from pyspark.ml.common import inherit_doc

from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import QuantileDiscretizer, VectorSlicer
from pyspark.ml.feature import ChiSqSelector

from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.feature import StringIndexer, OneHotEncoder,VectorAssembler
from pyspark.ml.feature import Binarizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

from pyspark.ml.stat import Correlation, ChiSquareTest
from pyspark.ml.stat import Summarizer

from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import NGram
from pyspark.ml.feature import PCA
from pyspark.ml.feature import PolynomialExpansion
from pyspark.ml.feature import DCT
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.feature import OneHotEncoder, Normalizer, StandardScaler
from pyspark.ml.feature import MinMaxScaler, MaxAbsScaler, Bucketizer
from pyspark.ml.feature import ElementwiseProduct
from pyspark.ml.feature import SQLTransformer

from pyspark.ml.recommendation import ALS
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.fpm import PrefixSpan

from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import GaussianMixture
from pyspark.ml.clustering import LDA
from pyspark.ml.clustering import BisectingKMeans
from pyspark.ml.clustering import PowerIterationClustering


from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import LogisticRegression , OneVsRest
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import FMClassifier

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.regression import AFTSurvivalRegression
from pyspark.ml.regression import IsotonicRegression
from pyspark.ml.regression import FMRegressor

from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from pyspark.mllib.fpm import FPGrowth
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.mllib.classification import LogisticRegressionModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import StreamingLinearRegressionWithSGD

In [2]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

21/07/26 07:14:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
#################################
df= spark.read.format('csv').option("header",'true').load("Machine_signal.csv")
df.show(2)

+----------+------+
|   machine|signal|
+----------+------+
|machine370|  A154|
|machine370|  B040|
+----------+------+
only showing top 2 rows



In [9]:
df.take(1)  # Take the first row

[Row(machine='machine370', signal='A154')]

In [10]:
df.printSchema()

root
 |-- machine: string (nullable = true)
 |-- signal: string (nullable = true)



In [11]:
# Create dataframe for FPGrowth model input
from pyspark.sql.functions import collect_list, col
from pyspark.sql import functions as F 
from pyspark.sql.functions import *
transactions = df.groupBy("machine")\
      .agg(F.collect_set("signal"))

transactions.show()

+----------+--------------------+
|   machine| collect_set(signal)|
+----------+--------------------+
|machine370|  [A154, B040, A173]|
|machine109|  [A133, B022, B055]|
|machine105|  [A133, B022, B055]|
| machine41|[A126, B055, C321...|
+----------+--------------------+



In [12]:
from pyspark.ml.fpm import FPGrowth
mod= FPGrowth(itemsCol="collect_set(signal)",minSupport=0.01, minConfidence=0.1)
#mod= FPGrowth(minSupport=0.01, minConfidence=0.1)
model = mod.fit(transactions)

model.freqItemsets.show(10, truncate=False)

+------------------+----+
|items             |freq|
+------------------+----+
|[B055]            |3   |
|[B022]            |2   |
|[B022, B055]      |2   |
|[A133]            |2   |
|[A133, B022]      |2   |
|[A133, B022, B055]|2   |
|[A133, B055]      |2   |
|[C321]            |1   |
|[C321, B055]      |1   |
|[A173]            |1   |
+------------------+----+
only showing top 10 rows



In [18]:
model.freqItemsets.sort('freq', ascending=False).show(10, truncate=False)

+------------------+----+
|items             |freq|
+------------------+----+
|[B055]            |3   |
|[B022, B055]      |2   |
|[B022]            |2   |
|[A133, B022, B055]|2   |
|[A133, B055]      |2   |
|[A133]            |2   |
|[A133, B022]      |2   |
|[C321]            |1   |
|[C321, B055]      |1   |
|[A173]            |1   |
+------------------+----+
only showing top 10 rows



In [14]:
model.freqItemsets.count()

28

In [15]:
model.associationRules.sort('confidence', ascending=False).show(10, truncate=False)

+------------+----------+----------+------------------+-------+
|antecedent  |consequent|confidence|lift              |support|
+------------+----------+----------+------------------+-------+
|[A133]      |[B022]    |1.0       |2.0               |0.5    |
|[A133]      |[B055]    |1.0       |1.3333333333333333|0.5    |
|[A126, C321]|[A121]    |1.0       |4.0               |0.25   |
|[A126, C321]|[B055]    |1.0       |1.3333333333333333|0.25   |
|[A154, A173]|[B040]    |1.0       |4.0               |0.25   |
|[A126, A121]|[C321]    |1.0       |4.0               |0.25   |
|[A126, A121]|[B055]    |1.0       |1.3333333333333333|0.25   |
|[A154, B040]|[A173]    |1.0       |4.0               |0.25   |
|[A121, C321]|[B055]    |1.0       |1.3333333333333333|0.25   |
|[A121, C321]|[A126]    |1.0       |4.0               |0.25   |
+------------+----------+----------+------------------+-------+
only showing top 10 rows



In [16]:
# transform examines the input items against all the association rules and summarise the
# consequents as prediction
model.transform(transactions).show()

+----------+--------------------+------------------+
|   machine| collect_set(signal)|        prediction|
+----------+--------------------+------------------+
|machine370|  [A154, B040, A173]|                []|
|machine109|  [A133, B022, B055]|[C321, A121, A126]|
|machine105|  [A133, B022, B055]|[C321, A121, A126]|
| machine41|[A126, B055, C321...|      [B022, A133]|
+----------+--------------------+------------------+

